In [8]:
import cPickle as pickle
import sys
sys.path.append('/usr/app/frustum-pointnets/train/')
import provider
import numpy as np
import copy


# load evaluation results

In [6]:
data_path = '/usr/app/frustum-pointnets/train/detection_results_pyun_debug2.pickle'
with open(data_path, 'rb') as fp:
    ps_list = pickle.load(fp)
    segp_list = pickle.load(fp)
    center_list = pickle.load(fp)
    heading_cls_list = pickle.load(fp)
    heading_res_list = pickle.load(fp)
    size_cls_list = pickle.load(fp)
    size_res_list = pickle.load(fp)
    rot_angle_list = pickle.load(fp)
    score_list = pickle.load(fp)
    onehot_list = pickle.load(fp)
    obj_xyz_list = pickle.load(fp)


# compute l1 norm between pc and recovered pc

In [7]:
diff_obj_xyz_list = []
reldiff_obj_xyz_list = []
for idx in range(len(ps_list)):
    # deep copy data,
    # otherwise provider.from_prediction_to_label_format will cause bug.
    ps = np.copy(ps_list[idx])
    segp = np.copy(segp_list[idx])
    obj_xyz = np.copy(obj_xyz_list[idx])
    rot_angle = np.copy(rot_angle_list[idx])
    center = np.copy(center_list[idx])
    heading_cls = np.copy(heading_cls_list[idx])
    heading_res = np.copy(heading_res_list[idx])
    size_cls = size_cls_list[idx]
    size_res = np.copy(size_res_list[idx])

    # rotate point cloud (frustrum frame) to camera frame
    ps = provider.rotate_pc_along_y(ps, -rot_angle)
    # mask off background points
    segp_mask = segp == 1
    ps_mask = ps[segp_mask]
    obj_xyz_mask = obj_xyz[segp_mask]
    # get box3d from prediction
    h,w,l,tx,ty,tz,ry = provider.from_prediction_to_label_format(center,
        heading_cls, heading_res,
        size_cls, size_res, rot_angle)
    # recover normalized obj_xyz_mask to camera frame
    obj_xyz_rcv = obj_xyz_mask * 2.0 - 1 
    obj_xyz_rcv = obj_xyz_rcv * np.array([l/2.0, h/2.0, w/2.0])
    obj_xyz_rcv = provider.rotate_pc_along_y(obj_xyz_rcv, - ry)
    obj_xyz_rcv = obj_xyz_rcv + np.array([tx, ty, tz])
    # compute l1 norm
    diff_obj_xyz = np.abs(ps_mask[:, :3] - obj_xyz_rcv)
    diff_obj_xyz_list.append(diff_obj_xyz)
    # compute relative diff
    reldiff_obj_xyz = diff_obj_xyz / np.array([l, h, w])
    reldiff_obj_xyz_list.append(reldiff_obj_xyz)
        
diff_obj_xyz_np = np.concatenate(diff_obj_xyz_list)
print("diff mean is {}".format(diff_obj_xyz_np.mean(axis=0)))
print("diff std is {}".format(diff_obj_xyz_np.std(axis=0)))
reldiff_obj_xyz_np = np.concatenate(reldiff_obj_xyz_list)
print("reldiff mean is {}".format(reldiff_obj_xyz_np.mean(axis=0)))
print("reldiff std is {}".format(reldiff_obj_xyz_np.std(axis=0)))

diff mean is [ 0.25544572  0.80706577  0.44073082]
diff std is [ 0.45202915  0.11794566  0.74970163]
reldiff mean is [ 0.10227204  0.5023732   0.36458132]
reldiff std is [ 0.17659302  0.06267772  0.65612802]


# visualization

In [ ]:
from open3d import *
idx = 0
# deep copy data,
# otherwise provider.from_prediction_to_label_format will cause bug.
ps = np.copy(ps_list[idx])
segp = np.copy(segp_list[idx])
obj_xyz = np.copy(obj_xyz_list[idx])
rot_angle = np.copy(rot_angle_list[idx])
center = np.copy(center_list[idx])
heading_cls = np.copy(heading_cls_list[idx])
heading_res = np.copy(heading_res_list[idx])
size_cls = size_cls_list[idx]
size_res = np.copy(size_res_list[idx])

# rotate point cloud (frustrum frame) to camera frame
ps = provider.rotate_pc_along_y(ps, -rot_angle)
# mask off background points
segp_mask = segp == 1
ps_mask = ps[segp_mask]
obj_xyz_mask = obj_xyz[segp_mask]
# get box3d from prediction
h,w,l,tx,ty,tz,ry = provider.from_prediction_to_label_format(center,
    heading_cls, heading_res,
    size_cls, size_res, rot_angle)
print(h,w,l,tx,ty,tz,ry)
# recover normalized obj_xyz_mask to camera frame
obj_xyz_rcv = obj_xyz_mask * 2.0 - 1 
obj_xyz_rcv = obj_xyz_rcv * np.array([l/2.0, h/2.0, w/2.0])
obj_xyz_rcv = provider.rotate_pc_along_y(obj_xyz_rcv, - ry)
obj_xyz_rcv = obj_xyz_rcv + np.array([tx, ty, tz])

print("center point is at{}".format(np.array([tx, ty, tz])))
print("box3d size is {}".format(np.array([l, h, w])))

print("ps shape is {}".format(ps.shape))
print("ps range is {} - {}".format(ps[:,:3].min(axis=0), ps[:, :3].max(axis=0)))
ps_pcd = PointCloud()
ps_pcd.points = Vector3dVector(ps[:, :3])
write_point_cloud("ps.pcd", ps_pcd)
print("ps_mask shape is {}".format(ps_mask.shape))
print("ps_mask range is {} - {}".format(ps_mask[:,:3].min(axis=0), ps_mask[:, :3].max(axis=0)))
ps_mask_pcd = PointCloud()
ps_mask_pcd.points = Vector3dVector(ps_mask[:, :3])
write_point_cloud("ps_mask.pcd", ps_mask_pcd)
print("obj_xyz_mask shape is {}".format(obj_xyz_mask.shape))
print("obj_xyz_mask range is {} - {}".format(obj_xyz_mask[:,:3].min(axis=0), obj_xyz_mask[:, :3].max(axis=0)))
obj_xyz_mask_pcd = PointCloud()
obj_xyz_mask_pcd.points = Vector3dVector(obj_xyz_mask[:, :3])
write_point_cloud("obj_xyz_mask.pcd", obj_xyz_mask_pcd)
print("obj_xyz_rcv shape is {}".format(obj_xyz_rcv.shape))
print("obj_xyz_rcv range is {} - {}".format(obj_xyz_rcv[:,:3].min(axis=0), obj_xyz_rcv[:, :3].max(axis=0)))
obj_xyz_rcv_pcd = PointCloud()
obj_xyz_rcv_pcd.points = Vector3dVector(obj_xyz_rcv[:, :3])
write_point_cloud("obj_xyz_rcv.pcd", obj_xyz_rcv_pcd)


In [4]:
len(ps_list)

25392